In [21]:
#These are the installs necessary to run the code.
#!pip install sierrapy
#!pip install biopython

#Other imports
%pylab inline
import pandas as pd
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
import os

Populating the interactive namespace from numpy and matplotlib


In [22]:
'''These functions make writing the necessary files much faster.'''

def write_fasta(f_name,seq,ID=''):
    with open(f_name+'.txt', "w") as out_handle:
        out_handle.write('>{}\n'.format(ID))
        out_handle.write(str(seq))
        
def write_fasta_multi(f_name,seqs,ID='unk'):
    with open(f_name+'.txt', "w") as out_handle:
        for seq in seqs:
            out_handle.write('>{}\n'.format(ID))
            out_handle.write(str(seq)+'\n')

def blast(f_name):
    #Read the FASTA file
    fasta_string = open(f_name+'.txt').read()
    print('Searching {}'.format(f_name)) #Progress marker
    #Submit FASTA file to BLASTn, store results
    result_handle = NCBIWWW.qblast("blastn", "nt", fasta_string)
    print('Done! Writing:') #Progress marker
    #Write the result as xml for parsing
    with open(f_name+".xml", "w") as out_handle:
         out_handle.write(result_handle.read())
    result_handle.close()
    print('Returning result for {}'.format(f_name))
    
    #Open resulting xml file
    result_handle = open(f_name+".xml")
    #Parse and store as blast record.
    blast_record = NCBIXML.read(result_handle)
    return blast_record

def rm_whitespace(s):
    return ''.join(s.split())

def sierra(seqs,response_type,gene_type):
    gene_type = rm_whitespace(gene_type)
    for i,seq in enumerate(seqs):
        try:
            #This creates a directory with the sequence index as it's title
            #Keeps the data organized.
            path = 'files/{}/{}/{}'.format(response_type,gene_type,i)
            os.makedirs(path)
            #Use string interpolation (the {} thing) to define generic output filename
            f_name = 'files/{}/{}/{}/{}_seq'.format(response_type,gene_type,i,i)
            ID = 'unk'
            #Make a new FASTA file, this time identified - 
            #We'll submit that to sierrapy
            write_fasta(f_name,seq,ID=ID)
            #Submit to sierrapy for results! -o specifies we want to store the result.
            !sierrapy fasta {f_name+'.txt'} -o {f_name+'.json'}
        except: 
            pass


In [23]:
#First, read in the data.
data = pd.concat([pd.read_csv('training_data.csv'),pd.read_csv('test_data.csv')])
data = data.dropna(how='any')

#Separate by response type:
response_types = [v for v in data['Resp'].unique() if type(v)==int]
gene_types = [c for c in data.columns if 'Seq' in c]
for response_type in response_types:
    for gene_type in gene_types:
        seqs = data[data['Resp']==response_type][gene_type].iloc[0:3]
        sierra(seqs,response_type,gene_type)

20it [00:02,  8.56it/s]                                                         
20it [00:01, 10.27it/s]                                                         
20it [00:01, 13.91it/s]                                                         
20it [00:08,  2.33it/s]                                                         
20it [00:01, 11.16it/s]                                                         
20it [00:01, 14.11it/s]                                                         
20it [00:01, 16.63it/s]                                                         
20it [00:01, 19.10it/s]                                                         
20it [00:02,  7.46it/s]                                                         
20it [00:01, 13.25it/s]                                                         
20it [00:01, 10.59it/s]                                                         
20it [00:01, 11.00it/s]                                                         


In [ ]:
'''
And that's it! Now you can take the mutations list, 
and the indices of each mutation, and find shared
mutations across the bad performers.
'''